In [1]:
# Install sentence-transformers
!pip install -q sentence-transformers

In [1]:
# Import libraries
from langchain_community.graphs import Neo4jGraph
from graph_rag_embeddings import EntityEmbeddings
import numpy as np

/home/duo/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NEO4J_URI="neo4j+s://efe5c094.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="GW7dZdXYlFaklMMPzmwOHBXSzt1g7zVPHDkB6wcnk5w"
NEO4J_DATABASE="neo4j"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print("Kết nối Neo4j thành công!")

/tmp/ipykernel_4994/1746938119.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


Kết nối Neo4j thành công!


## Init Embedding Model

In [3]:
embeddings = EntityEmbeddings(
    graph=graph,
    model_name="truro7/vn-law-embedding"
)

print(f"\nModel: {embeddings.model_name}")
print(f"Embedding dimension: {embeddings.embedding_dim}")

Loading embedding model: truro7/vn-law-embedding


Invalid model-index. Not loading eval results into CardData.
Invalid model-index. Not loading eval results into CardData.


Model loaded! Embedding dimension: 768

Model: truro7/vn-law-embedding
Embedding dimension: 768


## Check Current State

In [4]:
# Check embedding coverage trước khi generate
coverage = embeddings.check_embedding_coverage()

print("=== EMBEDDING COVERAGE ===")
print(f"Total entities: {coverage['total_entities']:,}")
print(f"Entities with embeddings: {coverage['entities_with_embeddings']:,}")
print(f"Coverage: {coverage['coverage_percent']:.1f}%")

print("\nCoverage by type:")
for row in coverage['coverage_by_type']:
    print(f"  {row['type']}: {row['with_embedding']}/{row['total']} ({row['coverage_percent']:.1f}%)")

Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `embedding` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=3, column=17, offset=35>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 35, 'line': 3, 'column': 17}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n        MATCH (n)\n        WHERE n.embedding IS NOT NULL\n        RETURN count(n) as count\n        '
Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `embedding` does not exist in database `neo4j`. Verify that the 

=== EMBEDDING COVERAGE ===
Total entities: 1,164
Entities with embeddings: 0
Coverage: 0.0%

Coverage by type:
  VIOLATION: 0/467 (0.0%)
  ROAD_ELEMENT: 0/251 (0.0%)
  VEHICLE: 0/247 (0.0%)
  PENALTY: 0/134 (0.0%)
  ORGANIZATION: 0/51 (0.0%)
  ACTION: 0/10 (0.0%)
  DOCUMENT: 0/3 (0.0%)
  ENTITY: 0/1 (0.0%)


## Generate Embeddings for All Entities

In [5]:
# Generate embeddings cho tất cả entities
stats = embeddings.generate_and_store_all_embeddings(batch_size=32)

print("\n=== STATISTICS ===")
print(f"Total entities: {stats['total']}")
print(f"Successfully embedded: {stats['success']}")
print(f"Failed: {stats['failed']}")
print(f"Embedding dimension: {stats['embedding_dim']}")

Starting embedding generation...


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=7, column=14, offset=177>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 177, 'line': 7, 'column': 14}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n         MATCH (n)\n         WHERE n.name IS NOT NULL\n         RETURN n.name as name,\n             labels(n)[0] as type,\n             n.description as description,\n             id(n) as node_id\n         ORDER BY n.name\n         '


Found 1164 entities in graph
Creating entity texts...
Generating embeddings for 1164 entities...


Batches: 100%|██████████| 37/37 [00:31<00:00,  1.17it/s]


Storing embeddings to Neo4j...


100%|██████████| 1164/1164 [01:24<00:00, 13.81it/s]


EMBEDDINGS GENERATED!
Success: 1164/1164
Failed: 0
Embedding dimension: 768

=== STATISTICS ===
Total entities: 1164
Successfully embedded: 1164
Failed: 0
Embedding dimension: 768


In [5]:
# Check và update embeddings cho entities mới
update_stats = embeddings.update_embeddings_for_new_entities()

print(f"\nNew embeddings generated: {update_stats['new_embeddings']}")
print(f"Total entities checked: {update_stats['total']}")

Found 808 entities without embeddings


Batches: 100%|██████████| 26/26 [00:23<00:00,  1.09it/s]


Updated 808 new embeddings

New embeddings generated: 808
Total entities checked: 808
